# Best Level OFI


In [2]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('../files/first_25000_rows.csv')
df['ts_recv'] = pd.to_datetime(df['ts_recv'])

print("Building Best-Level OFI...")
print("Step 2.1: Understanding the OFI Formula")

# The formula from the paper for Best-Level OFI:
# OFI = sum(bid_flow - ask_flow) over time interval
# Where:
# bid_flow = change in bid size when price stays same, or full size when price improves
# ask_flow = change in ask size when price stays same, or full size when price improves

print("\nLet's examine a few consecutive rows to see how order book changes:")

# Look at first 5 rows to understand the pattern
sample_data = df[['ts_recv', 'action', 'side', 'price', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00']].head(8)
print(sample_data)

print("\nNotice how bid_px_00, ask_px_00, bid_sz_00, ask_sz_00 change between rows")
print("This is what we use to calculate order flow")

Building Best-Level OFI...
Step 2.1: Understanding the OFI Formula

Let's examine a few consecutive rows to see how order book changes:
                              ts_recv action side   price  bid_px_00  \
0 2024-10-21 11:54:29.221230963+00:00      C    B  233.62     233.67   
1 2024-10-21 11:54:29.223936626+00:00      A    B  233.67     233.67   
2 2024-10-21 11:54:29.225196809+00:00      A    B  233.67     233.67   
3 2024-10-21 11:54:29.712600612+00:00      A    B  233.52     233.67   
4 2024-10-21 11:54:29.764839221+00:00      C    B  233.52     233.67   
5 2024-10-21 11:54:29.764851707+00:00      A    B  233.54     233.67   
6 2024-10-21 11:54:36.289594629+00:00      A    A  233.90     233.67   
7 2024-10-21 11:54:37.990960617+00:00      A    A  233.73     233.67   

   ask_px_00  bid_sz_00  ask_sz_00  
0     233.74        139        200  
1     233.74        141        200  
2     233.74        144        200  
3     233.74        144        200  
4     233.74        144       

In [4]:
# Step 2.2: Calculate order flows for each market update

def calculate_order_flows(df):
    df_copy = df.copy()
    df_copy = df_copy.sort_values('ts_recv').reset_index(drop=True)
    
    # Initialize flow columns
    df_copy['bid_flow'] = 0.0
    df_copy['ask_flow'] = 0.0
    
    # Calculate flows for each row (starting from second row)
    for i in range(1, len(df_copy)):
        current = df_copy.iloc[i]
        previous = df_copy.iloc[i-1]
        
        # Bid flow calculation
        if current['bid_px_00'] > previous['bid_px_00']:
            # Price improved (went up), count full size
            df_copy.loc[i, 'bid_flow'] = current['bid_sz_00']
        elif current['bid_px_00'] == previous['bid_px_00']:
            # Same price, count size change
            df_copy.loc[i, 'bid_flow'] = current['bid_sz_00'] - previous['bid_sz_00']
        elif current['bid_px_00'] < previous['bid_px_00']:
            # Price got worse (went down), count negative full size
            df_copy.loc[i, 'bid_flow'] = -previous['bid_sz_00']
            
        # Ask flow calculation  
        if current['ask_px_00'] < previous['ask_px_00']:
            # Price improved (went down), count full size
            df_copy.loc[i, 'ask_flow'] = current['ask_sz_00']
        elif current['ask_px_00'] == previous['ask_px_00']:
            # Same price, count size change
            df_copy.loc[i, 'ask_flow'] = current['ask_sz_00'] - previous['ask_sz_00']
        elif current['ask_px_00'] > previous['ask_px_00']:
            # Price got worse (went up), count negative full size
            df_copy.loc[i, 'ask_flow'] = -previous['ask_sz_00']
    
    return df_copy

# Calculate the flows
df_with_flows = calculate_order_flows(df)

print("Sample of calculated flows:")
sample_flows = df_with_flows[['ts_recv', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_flow', 'ask_flow']].head(10)
print(sample_flows)

Sample of calculated flows:
                              ts_recv  bid_px_00  ask_px_00  bid_sz_00  \
0 2024-10-21 11:54:29.221230963+00:00     233.67     233.74        139   
1 2024-10-21 11:54:29.223936626+00:00     233.67     233.74        141   
2 2024-10-21 11:54:29.225196809+00:00     233.67     233.74        144   
3 2024-10-21 11:54:29.712600612+00:00     233.67     233.74        144   
4 2024-10-21 11:54:29.764839221+00:00     233.67     233.74        144   
5 2024-10-21 11:54:29.764851707+00:00     233.67     233.74        144   
6 2024-10-21 11:54:36.289594629+00:00     233.67     233.74        144   
7 2024-10-21 11:54:37.990960617+00:00     233.67     233.73        144   
8    2024-10-21 11:54:39.124458+00:00     233.67     233.75        144   
9 2024-10-21 11:54:39.134307134+00:00     233.67     233.74        144   

   ask_sz_00  bid_flow  ask_flow  
0        200       0.0       0.0  
1        200       2.0       0.0  
2        200       3.0       0.0  
3        200     

In [6]:
# Step 2.3: Calculate OFI over time intervals

def calculate_best_level_ofi(df_flows, time_window_seconds=60):
    df_flows['ofi_instant'] = df_flows['bid_flow'] - df_flows['ask_flow']
    
    # Set timestamp as index for resampling
    df_flows_indexed = df_flows.set_index('ts_recv')
    
    # Aggregate OFI over time windows
    ofi_aggregated = df_flows_indexed['ofi_instant'].resample(f'{time_window_seconds}s').sum().reset_index()
    ofi_aggregated.columns = ['timestamp', 'best_level_ofi']
    
    return ofi_aggregated

# Calculate OFI with 1-minute windows
ofi_1min = calculate_best_level_ofi(df_with_flows, time_window_seconds=60)

print(f"Best-Level OFI calculated over 1-minute intervals:")
print(f"Number of time intervals: {len(ofi_1min)}")
print("\nFirst 10 OFI values:")
print(ofi_1min.head(10))

print(f"\nOFI Statistics:")
print(f"Mean: {ofi_1min['best_level_ofi'].mean():.2f}")
print(f"Std: {ofi_1min['best_level_ofi'].std():.2f}")
print(f"Min: {ofi_1min['best_level_ofi'].min():.2f}")
print(f"Max: {ofi_1min['best_level_ofi'].max():.2f}")

Best-Level OFI calculated over 1-minute intervals:
Number of time intervals: 71

First 10 OFI values:
                  timestamp  best_level_ofi
0 2024-10-21 11:54:00+00:00          -195.0
1 2024-10-21 11:55:00+00:00          -916.0
2 2024-10-21 11:56:00+00:00           199.0
3 2024-10-21 11:57:00+00:00           201.0
4 2024-10-21 11:58:00+00:00           863.0
5 2024-10-21 11:59:00+00:00           208.0
6 2024-10-21 12:00:00+00:00           498.0
7 2024-10-21 12:01:00+00:00          -199.0
8 2024-10-21 12:02:00+00:00          -600.0
9 2024-10-21 12:03:00+00:00          -399.0

OFI Statistics:
Mean: -169.49
Std: 761.90
Min: -4344.00
Max: 1380.00


In [7]:
# Step 2.4: Test different time windows

time_windows = [30, 60, 120, 300]  # 30sec, 1min, 2min, 5min

ofi_results = {}

for window in time_windows:
    ofi_result = calculate_best_level_ofi(df_with_flows, time_window_seconds=window)
    ofi_results[f'{window}s'] = ofi_result
    
    print(f"\n{window}-second window OFI:")
    print(f"  Intervals: {len(ofi_result)}")
    print(f"  Mean: {ofi_result['best_level_ofi'].mean():.2f}")
    print(f"  Std: {ofi_result['best_level_ofi'].std():.2f}")
    print(f"  Range: [{ofi_result['best_level_ofi'].min():.2f}, {ofi_result['best_level_ofi'].max():.2f}]")

# Show sample values for different windows
print(f"\nSample OFI values across different time windows:")
print("Timestamp               | 30s   | 60s   | 120s  | 300s")
print("-----------------------|-------|-------|-------|-------")

max_rows = min(5, len(ofi_results['30s']))
for i in range(max_rows):
    ts = ofi_results['30s'].iloc[i]['timestamp'].strftime('%H:%M:%S')
    v30 = ofi_results['30s'].iloc[i]['best_level_ofi']
    
    # Find corresponding values in other windows
    v60 = ofi_results['60s'][ofi_results['60s']['timestamp'] <= ofi_results['30s'].iloc[i]['timestamp']]['best_level_ofi'].iloc[-1] if len(ofi_results['60s']) > 0 else 0
    v120 = ofi_results['120s'][ofi_results['120s']['timestamp'] <= ofi_results['30s'].iloc[i]['timestamp']]['best_level_ofi'].iloc[-1] if len(ofi_results['120s']) > 0 else 0
    v300 = ofi_results['300s'][ofi_results['300s']['timestamp'] <= ofi_results['30s'].iloc[i]['timestamp']]['best_level_ofi'].iloc[-1] if len(ofi_results['300s']) > 0 else 0
    
    print(f"{ts}               | {v30:5.0f} | {v60:5.0f} | {v120:5.0f} | {v300:5.0f}")


30-second window OFI:
  Intervals: 141
  Mean: -85.35
  Std: 500.22
  Range: [-3456.00, 1180.00]

60-second window OFI:
  Intervals: 71
  Mean: -169.49
  Std: 761.90
  Range: [-4344.00, 1380.00]

120-second window OFI:
  Intervals: 36
  Mean: -334.28
  Std: 1152.51
  Range: [-4163.00, 1214.00]

300-second window OFI:
  Intervals: 15
  Mean: -802.27
  Std: 2227.00
  Range: [-6493.00, 3150.00]

Sample OFI values across different time windows:
Timestamp               | 30s   | 60s   | 120s  | 300s
-----------------------|-------|-------|-------|-------
11:54:00               |     5 |  -195 | -1111 |  -195
11:54:30               |  -200 |  -195 | -1111 |  -195
11:55:00               |  -200 |  -916 | -1111 |   555
11:55:30               |  -716 |  -916 | -1111 |   555
11:56:00               |   199 |   199 |   400 |   555


In [8]:
# Step 2.5: Validation - examine specific cases

print("Validation: Let's trace through a few examples manually")

# Look at cases where we have significant order flow
significant_flows = df_with_flows[abs(df_with_flows['ofi_instant']) > 50].head(5)

print("\nCases with significant instantaneous OFI:")
validation_cols = ['ts_recv', 'action', 'side', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_flow', 'ask_flow', 'ofi_instant']
print(significant_flows[validation_cols])

print(f"\nInterpretation:")
print(f"- Positive OFI = More buying pressure (bid flow > ask flow)")
print(f"- Negative OFI = More selling pressure (ask flow > bid flow)")
print(f"- Large absolute values = Significant imbalance")

# Summary statistics
print(f"\nFinal Best-Level OFI Summary (1-minute intervals):")
final_ofi = ofi_results['60s']
print(f"Time range: {final_ofi['timestamp'].min()} to {final_ofi['timestamp'].max()}")
print(f"Total intervals: {len(final_ofi)}")
print(f"Non-zero intervals: {(final_ofi['best_level_ofi'] != 0).sum()}")
print(f"Positive OFI intervals: {(final_ofi['best_level_ofi'] > 0).sum()}")
print(f"Negative OFI intervals: {(final_ofi['best_level_ofi'] < 0).sum()}")

# Save the result for next steps
final_ofi.to_csv('best_level_ofi.csv', index=False)
print(f"\nBest-Level OFI saved to 'best_level_ofi.csv'")

Validation: Let's trace through a few examples manually

Cases with significant instantaneous OFI:
                               ts_recv action side  bid_px_00  ask_px_00  \
7  2024-10-21 11:54:37.990960617+00:00      A    A     233.67     233.73   
8     2024-10-21 11:54:39.124458+00:00      C    A     233.67     233.75   
9  2024-10-21 11:54:39.134307134+00:00      A    A     233.67     233.74   
35 2024-10-21 11:55:26.448145163+00:00      A    A     233.67     233.73   
45 2024-10-21 11:55:34.079427496+00:00      C    B     233.67     233.73   

    bid_sz_00  ask_sz_00  bid_flow  ask_flow  ofi_instant  
7         144        200       0.0     200.0       -200.0  
8         144          1       0.0    -200.0        200.0  
9         144        200       0.0     200.0       -200.0  
35        144        200       0.0     200.0       -200.0  
45          5        200    -139.0       0.0       -139.0  

Interpretation:
- Positive OFI = More buying pressure (bid flow > ask flow)
- Negat